# SEIRHVD Data Fit

This Jupyter notebooks implements the SEIRHVD model for fitting the actual Chilean data in order to make projections in VMI and bed usage, amount of infected and deaths. This model uses the SEIRHVD 4.1 version. A visual representation of the model is available at: [SEIRHVD Miró Model](https://miro.com/app/board/o9J_ktzN4xA=/)

This implements a single instance to facilitate understanding the model prior to run multiple data fittings at a time.



In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
from numpy import linalg as LA

import platform
OS = platform.system()

import matplotlib.pyplot as plt
if OS == 'Linux':    
    %matplotlib tk
    print('Linux')
elif OS == 'Windows':
    %matplotlib qt
    print('Windows')
else:
    print('OS not detected :-|')

import sys
from pathlib import Path
sys.path.insert(1, '../src/SEIRHVD/')
sys.path.insert(1, '../src/utils/')
sys.path.insert(1, 'src/SEIRHVD/')
sys.path.insert(1, 'src/utils/')

from class_SEIRHUVD4 import SEIRHVD 
from Quarantine import Quarantine
from importdata import ImportData

Linux


## Time Variables
Declare Initial date for simulating and data fitting

In [2]:
# First simulation Initial date
initdate = datetime(2020,5,15)
# Date for change in SeroPrevalence
initdate2 = datetime(2020,8,24)

# Current date
currentdate = datetime.now()
currentday = (currentdate - initdate).days

## Import Data
To import data we use an ImportData object which is initialized with the region's cut and the initial date. RM region is represented by cut = 13.

In [3]:
tstate = '13'

# Import Data
RM = ImportData(tstate=tstate,initdate = initdate)
RM.importdata()

Importing General Data
Importing Population
Importing Active infected
Importing Accumulated Infected
Importing Daily Infected
Importing Sochimi Data 2
Importing Accumulated Deaths
Importing Active Infected by Minciencia
Importing Hospitalized/NonHospitalized Deaths
Done


## Datos de Hospitalizados a fitear

In [4]:
#RM.Hr = np.array(RM.UCI) + np.array(RM.UTI)
#RM.Hr_tot = np.array(RM.UCI_tot) + np.array(RM.UTI_tot)
RM.Hr = np.array(RM.UTI)
RM.Hr_tot = np.array(RM.UTI_tot)

## Simulation Parameters

In [5]:
# Total simulation time
tsimtot = 1000

tsim1 = (initdate2 - initdate).days 
tsim2 = tsimtot-tsim1

beta = 0.13
mu = 1.2
k = 0

SeroPrevFactor1=0.06
SeroPrevFactor2=0.025

expinfection=0 # Proportion in which the exposed infect - 0: nothing, 1: equally as Infected

## Quarantines
Build Quarantine Object:
 
     Q = Quarantine(rem_mob,max_mob=max_mob,qp=0,iqt=0,fqt=1000,movfunct = 'once')
     alpha = Q.alpha
     
     Plot Quarantine dynamics:
         Q.plot()

In [6]:
# Quarantines 
max_mob = 0.55
rem_mob = 0.45

Q1 = Quarantine(rem_mob,max_mob,qp=0,iqt=0,fqt=tsim1)
Q2 = Quarantine(rem_mob)

alpha = Q2.alpha

## Underreport
Fraction of Infected detected/reported  

$Imi_{det}$: Fraction of Mild detected   
$Ias_{det}$: fraction of asymptomatic detected  

If both are 1, means that all infected are detected so 
\begin{align}
I_{det} = I
\end{align}

In [7]:
Imi_det = 1
Ias_det = 0.5

## State Parameters:
The following parameters determine transitions probability and duration between different states. The values presented here are the default values set in the model. You can change them as it's showed in the following executable lines.

pE_Ias = 0.4  # Transition from exposed to Asymptomatic Infected  
tE_Ias = 5.0

pE_Imi = 0.55 # Transition from exposed to  Mild Infected  
tE_Imi = 5.0

pE_Icr = 0.01666 # Transition from exposed to  Critical Infected  
tE_Icr = 3.0

pE_Ise = 0.03334 # Transition from exposed to  Serious Infected  
tE_Ise = 3.0

pIas_R = 1.0   # Transition from Asymptomatic Infected to Recovered  
tIas_R = 10.0

pImi_R = 1.0  # Transition from Mild Infected to Recovered  
tImi_R = 15.0

pIse_Hse = 1.0 # Transition from Serious Infected to Serious Hospitalized (When Hospital capacity is not saturated)  
tIse_Hse = 3.0 

pIse_D = 1.0  # Transition from Serious Infected to Death (When Hospital capacity is saturated)  
tIse_D = 3.0         

pIcr_V = 1.0  # Transition from Critical Infected to Ventilator (When Ventilators capacity is not saturated)  
tIcr_V = 3.0 

pIcr_D = 1.0  # Transition from Serious Infected to Death (When Ventilators capacity is saturated)  
tIcr_D = 3.0         

pHse_R = 0.97 # Transition from Serious Hospitalized to Recovered  
tHse_R = 11.0

pHse_V = 0.03 # Transition from Serious Hospitalized to Ventilators (When Ventilators capacity is not saturated)  
tHse_V = 3.0

pHse_D = 0.03 # Transition from Serious Hospitalized to Death (When Ventilators capacity is saturated)  
tHse_D = 3.0        

pV_Hout = 0.5  # Transition from Ventilators to Hospital Recovery (Hout)   
tV_Hout = 15.0

pV_D = 0.5 # Transition from Ventilators to Death  
tV_D = 15.0

pHout_R = 1.0 # Transition from Hospital Recovery (Hout) to Recovered  
tHout_R = 4.0

pD_B = 1.0 # Transition from Dead to buried  
tD_B = 1.0 

betaD = 0 # Contagion by deads rate  
eta = 0.0 # Immunity loss rate  

In [8]:
pE_Ias = 0.414  # Transition from exposed to Asymptomatic Infected  
tE_Ias = 5.0

pE_Imi = 0.55 # Transition from exposed to  Mild Infected  
tE_Imi = 5.0

pE_Icr = 0.016 # Transition from exposed to  Critical Infected  
tE_Icr = 3.0

pE_Ise = 0.02 # Transition from exposed to  Serious Infected  
tE_Ise = 3.0

pIas_R = 1.0   # Transition from Asymptomatic Infected to Recovered  
tIas_R = 10.0

pImi_R = 1.0  # Transition from Mild Infected to Recovered  
tImi_R = 15.0

pIse_Hse = 1.0 # Transition from Serious Infected to Serious Hospitalized (When Hospital capacity is not saturated)  
tIse_Hse = 3.0 

pIse_D = 1.0  # Transition from Serious Infected to Death (When Hospital capacity is saturated)  
tIse_D = 3.0         

pIcr_V = 1.0  # Transition from Critical Infected to Ventilator (When Ventilators capacity is not saturated)  
tIcr_V = 3.0 

pIcr_D = 1.0  # Transition from Serious Infected to Death (When Ventilators capacity is saturated)  
tIcr_D = 3.0         

pHse_R = 0.97 # Transition from Serious Hospitalized to Recovered  
tHse_R = 11.0

pHse_V = 0.03 # Transition from Serious Hospitalized to Ventilators (When Ventilators capacity is not saturated)  
tHse_V = 3.0

pHse_D = 0.03 # Transition from Serious Hospitalized to Death (When Ventilators capacity is saturated)  
tHse_D = 3.0        

pV_Hout = 0.5  # Transition from Ventilators to Hospital Recovery (Hout)   
tV_Hout = 12.0

pV_D = 0.5 # Transition from Ventilators to Death  
tV_D = 12.0

pHout_R = 1.0 # Transition from Hospital Recovery (Hout) to Recovered  
tHout_R = 4.0

# Simulation

## Initial SeroPrev period
Initialize Simulation Object, set params and run the simulation. 

In [9]:
simulation1 = SEIRHVD(tsim1,beta,mu,alpha,k=k,SeroPrevFactor=SeroPrevFactor1,expinfection=expinfection,RealIC = RM,Imi_det = Imi_det,Ias_det = Ias_det)

InitialCondition Object Data


### Setting new params

In [10]:
simulation1.pE_Ias=pE_Ias
simulation1.tE_Ias=tE_Ias
simulation1.pE_Imi=pE_Imi
simulation1.tE_Imi=tE_Imi
simulation1.pE_Icr=pE_Icr
simulation1.tE_Icr=tE_Icr
simulation1.pE_Ise=pE_Ise
simulation1.tE_Ise=tE_Ise
simulation1.pIas_R=pIas_R
simulation1.tIas_R =tIas_R
simulation1.pImi_R=pImi_R
simulation1.tImi_R =tImi_R
simulation1.pIse_Hse=pIse_Hse
simulation1.tIse_Hse=tIse_Hse
simulation1.pIse_D=pIse_D
simulation1.tIse_D=tIse_D
simulation1.pIcr_V=pIcr_V
simulation1.tIcr_V=tIcr_V
simulation1.pIcr_D=pIcr_D
simulation1.tIcr_D=tIcr_D
simulation1.pHse_R=pHse_R
simulation1.tHse_R =tHse_R
simulation1.pHse_V=pHse_V
simulation1.tHse_V=tHse_V
simulation1.pHse_D=pHse_D
simulation1.tHse_D=tHse_D
simulation1.pV_Hout=pV_Hout
simulation1.tV_Hout =tV_Hout
simulation1.pV_D=pV_D
simulation1.tV_D =tV_D
simulation1.pHout_R=pHout_R
simulation1.tHout_R=tHout_R
simulation1.setnewparams()

Compartimental model State parameters changed


## Run simulation

In [11]:
simulation1.integr_sci(0,tsim1,0.1)

  message: 'The solver successfully reached the end of the integration interval.'
     nfev: 2704
     njev: 57
      nlu: 57
      sol: None
   status: 0
  success: True
        t: array([0.00000000e+00, 2.27595382e-08, 4.55190764e-08, 2.27640901e-04,
       4.55236283e-04, 6.82831665e-04, 2.95878549e-03, 5.23473931e-03,
       7.51069313e-03, 9.78664695e-03, 3.25461852e-02, 5.53057234e-02,
       7.80652616e-02, 1.00824800e-01, 1.23584338e-01, 3.51179720e-01,
       5.78775102e-01, 8.06370484e-01, 1.03396587e+00, 1.26156125e+00,
       1.48915663e+00, 1.93528457e+00, 2.38141251e+00, 2.82754046e+00,
       3.27366840e+00, 3.71979634e+00, 4.26945143e+00, 4.81910652e+00,
       5.36876161e+00, 5.91841670e+00, 6.47875748e+00, 7.03909825e+00,
       7.59943903e+00, 8.48999967e+00, 9.38056031e+00, 1.02711209e+01,
       1.19636104e+01, 1.36560999e+01, 1.53485894e+01, 1.57717118e+01,
       1.61948342e+01, 1.66179566e+01, 1.67237372e+01, 1.68295178e+01,
       1.69352983e+01, 1.69617435e+01

## Simulation 2
Second SeroPrev period

In [12]:
simulation2 = SEIRHVD(tsim2,beta,mu,alpha,k=k,SeroPrevFactor=SeroPrevFactor2,Imi_det = Imi_det,Ias_det = Ias_det,SimIC=simulation1,initdate=initdate2)

In [13]:
simulation2.pE_Ias=pE_Ias
simulation2.tE_Ias=tE_Ias
simulation2.pE_Imi=pE_Imi
simulation2.tE_Imi=tE_Imi
simulation2.pE_Icr=pE_Icr
simulation2.tE_Icr=tE_Icr
simulation2.pE_Ise=pE_Ise
simulation2.tE_Ise=tE_Ise
simulation2.pIas_R=pIas_R
simulation2.tIas_R =tIas_R
simulation2.pImi_R=pImi_R
simulation2.tImi_R =tImi_R
simulation2.pIse_Hse=pIse_Hse
simulation2.tIse_Hse=tIse_Hse
simulation2.pIse_D=pIse_D
simulation2.tIse_D=tIse_D
simulation2.pIcr_V=pIcr_V
simulation2.tIcr_V=tIcr_V
simulation2.pIcr_D=pIcr_D
simulation2.tIcr_D=tIcr_D
simulation2.pHse_R=pHse_R
simulation2.tHse_R =tHse_R
simulation2.pHse_V=pHse_V
simulation2.tHse_V=tHse_V
simulation2.pHse_D=pHse_D
simulation2.tHse_D=tHse_D
simulation2.pV_Hout=pV_Hout
simulation2.tV_Hout =tV_Hout
simulation2.pV_D=pV_D
simulation2.tV_D =tV_D
simulation2.pHout_R=pHout_R
simulation2.tHout_R=tHout_R
simulation2.setnewparams()

Compartimental model State parameters changed


### Run simulation 2

In [14]:
simulation2.integr_sci(0,tsim2,0.1)

  message: 'The solver successfully reached the end of the integration interval.'
     nfev: 845
     njev: 15
      nlu: 15
      sol: None
   status: 0
  success: True
        t: array([0.00000000e+00, 1.62577443e-02, 3.25154886e-02, 1.63049623e-01,
       2.93583757e-01, 4.24117890e-01, 6.96297387e-01, 9.68476883e-01,
       1.24065638e+00, 1.51283587e+00, 1.97154947e+00, 2.43026306e+00,
       2.88897665e+00, 3.34769024e+00, 3.88923079e+00, 4.43077134e+00,
       4.97231188e+00, 5.51385243e+00, 6.04786901e+00, 6.58188559e+00,
       7.11590216e+00, 8.01308052e+00, 8.91025888e+00, 9.80743724e+00,
       1.06008028e+01, 1.13941684e+01, 1.21875340e+01, 1.39303953e+01,
       1.43661106e+01, 1.48018259e+01, 1.52375413e+01, 1.57107134e+01,
       1.61838855e+01, 1.66570577e+01, 1.77807558e+01, 1.89044539e+01,
       2.00281520e+01, 2.07182526e+01, 2.14083532e+01, 2.20984537e+01,
       2.27689155e+01, 2.34393773e+01, 2.41098390e+01, 2.48685553e+01,
       2.56272717e+01, 2.63859880e+01,

# Analysis

## Grid Plot

In [15]:
# Days to plot
days = 250
t_end = tsim1

In [17]:
fig, axs = plt.subplots(2, 3)
axs[0,0].set_xlim(0,days)
axs[0,0].scatter(RM.tr,RM.Ir,label='Real Active Data')
axs[0,0].plot(simulation1.t,simulation1.I_det,label='Infected')
axs[0,0].plot(simulation2.t+tsim1,simulation2.I_det,label='Infected')
axs[0,0].set_title('Active Infected')
axs[0,0].axvline(x = t_end, linestyle = 'dotted',color = 'grey')


axs[0,1].set_xlim(0,days)
axs[0,1].plot(simulation2.t+tsim1,simulation2.I_d_det,label='sim2')
axs[0,1].plot(simulation1.t,simulation1.I_d_det,label='sim1')
axs[0,1].scatter(RM.I_d_r_tr,RM.I_d_r,label='Daily Real Data')
axs[0,1].set_title('Daily infected')
axs[0,1].axvline(x = t_end, linestyle = 'dotted',color = 'grey')


axs[0,2].set_xlim(0,days)
axs[0,2].plot(simulation2.t+tsim1,simulation2.I_ac_det,label='sim2')
axs[0,2].plot(simulation1.t,simulation1.I_ac_det,label='sim1')
axs[0,2].scatter(RM.I_ac_r_tr,RM.I_ac_r,label='Accumulated Real Data')
axs[0,2].set_title('Accumulated Infected')
axs[0,2].axvline(x = t_end, linestyle = 'dotted',color = 'grey')


axs[1,0].set_xlim(0,days)
axs[1,0].plot(simulation2.t+tsim1,simulation2.B,label='sim2')
axs[1,0].plot(simulation1.t,simulation1.B,label='sim1')
axs[1,0].scatter(RM.hosp_tr,RM.Br_hosp,label='Real Data')
axs[1,0].set_title('Hospitalized Acmulated Deaths')
axs[1,0].axvline(x = t_end, linestyle = 'dotted',color = 'grey')


axs[1,1].set_xlim(0,days)
axs[1,1].plot(simulation1.t,simulation1.Hse+simulation1.Hout,label='UTI Beds')
axs[1,1].plot(simulation2.t+tsim1,simulation2.Hse+simulation2.Hout,label='UTI Beds')
axs[1,1].scatter(RM.sochimi_tr,RM.UTI,label='Real Data')
axs[1,1].plot(simulation1.t,simulation1.H_cap,label='Capacity', color = 'red')
axs[1,1].plot(simulation2.t+tsim1,simulation2.H_cap,label='Capacity', color = 'red')
axs[1,1].set_title('UTI Usage')
axs[1,1].axvline(x = t_end, linestyle = 'dotted',color = 'grey')


axs[1,2].set_xlim(0,days)
axs[1,2].plot(simulation2.t+tsim1,simulation2.V,label='VSat sim2')
axs[1,2].plot(simulation1.t,simulation1.V,label='Vsat sim1')
axs[1,2].scatter(RM.sochimi_tr,RM.Vr,label='Real Data')
axs[1,2].plot(simulation1.t,simulation1.V_cap,label='Capacity Data', color = 'red')
axs[1,2].plot(simulation2.t+tsim1,simulation2.V_cap,label='Capacity Data', color = 'red')
axs[1,2].set_title('VMI Usage')
axs[1,2].axvline(x = t_end, linestyle = 'dotted',color = 'grey')


#axs[2,1].plot(simulation2.t+tsim1,simulation2.V_D_d,label='V_D sim2')
#axs[2,1].plot(simulation2.t+tsim1,simulation2.Hse_D_d,label='Hse_D sim2')
#axs[2,1].plot(simulation2.t+tsim1,simulation2.Icr_D_d,label='Icr_D sim2')
#axs[2,1].plot(simulation2.t+tsim1,simulation2.Ise_D_d,label='Ise_D sim2')
#axs[2,1].plot(simulation1.t,simulation1.V_D_d,label='V_D sim1')
#axs[2,1].plot(simulation1.t,simulation1.Hse_D_d,label='Hse_D sim1')
#axs[2,1].plot(simulation1.t,simulation1.Icr_D_d,label='Icr_D sim1')
#axs[2,1].plot(simulation1.t,simulation1.Ise_D_d,label='Ise_D sim1')
#axs[2,1].set_title('Death by cause')
#axs[2,1].axvline(x = t_end, linestyle = 'dotted',color = 'grey')


## Single plots

### Daily new Infected

In [ ]:
plt.plot(simulation2.t+tsim1,simulation2.I_d_det,label='sim2')
plt.plot(simulation1.t,simulation1.I_d_det,label='sim1')
plt.legend(loc=0)
plt.title('Daily New Infected')
plt.show()

### Daily Deaths

In [ ]:
plt.plot(simulation2.t+tsim1,simulation2.D,label='sim2')
plt.plot(simulation1.t,simulation1.D,label='sim1')
plt.legend(loc=0)
plt.title('Deaths')
plt.show()

### VMI Usage

In [ ]:
plt.plot(simulation2.t+tsim1,simulation2.V_cap,label='VMI capacity sim2')
plt.plot(simulation1.t,simulation1.V_cap,label='VMI capacity sim1')
plt.plot(simulation1.t,simulation1.V,label='VMI sim1')
plt.plot(simulation2.t+tsim1,simulation2.V,label='VMI sim2')
plt.legend(loc=0)
plt.title('VMI')
plt.scatter(RM.sochimi_tr,RM.Vr,label='Real Data')
plt.scatter(RM.sochimi_tr,RM.Vr_tot,label='Real Data')
plt.show()

### Hospitalization

In [ ]:
plt.plot(simulation2.t+tsim1,simulation2.H_cap,label='sim2')
plt.plot(simulation1.t,simulation1.H_cap,label='sim1')
plt.plot(simulation1.t,simulation1.Hse+simulation1.Hout,label='UCI/UTI Beds')
plt.plot(simulation2.t+tsim1,simulation2.Hse+simulation2.Hout,label='UCI/UTI Beds')
plt.legend(loc=0)
plt.title('Hospitalization')

plt.scatter(RM.sochimi_tr,RM.Hr,label='Real Data')
plt.scatter(RM.sochimi_tr,RM.Hr_tot,label='Real Data')
plt.show()

### Deaths by cause

In [ ]:
plt.plot(simulation2.t+tsim1,simulation2.D,label='sim2')
plt.plot(simulation1.t,simulation1.D,label='sim1')


plt.plot(simulation2.t+tsim1,simulation2.V_D_d,label='V_D sim2')
plt.plot(simulation2.t+tsim1,simulation2.Hse_D_d,label='Hse_D sim2')
plt.plot(simulation2.t+tsim1,simulation2.Icr_D_d,label='Icr_D sim2')
plt.plot(simulation2.t+tsim1,simulation2.Ise_D_d,label='Ise_D sim2')

plt.plot(simulation1.t,simulation1.V_D_d,label='V_D sim1')
plt.plot(simulation1.t,simulation1.Hse_D_d,label='Hse_D sim1')
plt.plot(simulation1.t,simulation1.Icr_D_d,label='Icr_D sim1')
plt.plot(simulation1.t,simulation1.Ise_D_d,label='Ise_D sim1')

plt.legend(loc=0)
plt.title('Deaths by cause')
plt.show()

### Hospital saturation

In [ ]:
plt.plot(simulation2.t+tsim1,simulation2.H_sat,label='HSat sim2')
plt.plot(simulation1.t,simulation1.H_sat,label='HSat sim1')
plt.plot(simulation2.t+tsim1,simulation2.V_sat,label='VSat sim2')
plt.plot(simulation1.t,simulation1.V_sat,label='Vsat sim1')
plt.legend(loc=0)
plt.title('Saturation')
plt.show()

# Error Calculation
The following plots will show the simulation results with the real data

## Peak Date

In [ ]:
peakidx = np.where(RM.Ir==max(RM.Ir))[0][0]
datapeakdate = RM.sochimi_dates[peakidx]

In [ ]:
simulation.peak_date

In [ ]:
datapeakdate

In [ ]:
Peak_delta = (simulation.peak_date -datapeakdate).days

In [ ]:
Peak_delta

## Errors

In [ ]:
simulation = simulation1
t_end = tsim1
t_end_idx = np.where(np.array(RM.tr)>=t_end)[0][0]
idx = np.searchsorted(simulation.t,RM.tr[:t_end_idx])
E_I = np.sum(abs(RM.Ir[:t_end_idx]-simulation.I_det[idx]))/(np.mean(RM.Ir[:t_end_idx])*t_end_idx)

t_end_idx = np.where(np.array(RM.I_d_r_tr)>=t_end)[0][0]
idx = np.searchsorted(simulation.t,RM.I_d_r_tr[:t_end_idx])
E_Id = np.sum(abs(RM.I_d_r[:t_end_idx]-simulation.I_d_det[idx]))/(np.mean(RM.I_d_r[:t_end_idx])*t_end_idx)

t_end_idx = np.where(np.array(RM.I_ac_r_tr)>=t_end)[0][0]
idx = np.searchsorted(simulation.t,RM.I_ac_r_tr[:t_end_idx])
E_Iac = np.sum(abs(RM.I_ac_r[:t_end_idx]-simulation.I_ac_det[idx]))/(np.mean(RM.I_ac_r[:t_end_idx])*t_end_idx)

t_end_idx = np.where(np.array(RM.Br_tr)>=t_end)[0][0]
idx = np.searchsorted(simulation.t,RM.Br_tr[:t_end_idx])
E_D = np.sum(abs(RM.Br[:t_end_idx]-simulation.B[idx]))/(np.mean(RM.Br[:t_end_idx])*t_end_idx)

t_end_idx = np.where(np.array(RM.sochimi_tr)>=t_end)[0][0]
idx = np.searchsorted(simulation.t,RM.sochimi_tr[:t_end_idx])
E_UCI = np.sum(abs(RM.Hr[:t_end_idx]-(simulation.Hse[idx]+simulation.Hout[idx])))/(np.mean(RM.Hr[:t_end_idx])*t_end_idx)

t_end_idx = np.where(np.array(RM.sochimi_tr)>=t_end)[0][0]
idx = np.searchsorted(simulation.t,RM.sochimi_tr[:t_end_idx])
E_VMI = np.sum(abs(RM.Vr[:t_end_idx]-simulation.V[idx]))/(np.mean(RM.Vr[:t_end_idx])*t_end_idx)

Err = {'Active Infected':E_I,'Daily Infected':E_Id,'Accumulated Infected':E_Iac,'Deaths':E_D,'UCI/UTI':E_UCI,'VMI':E_VMI}


In [ ]:
Err = {'Active Infected':E_I,'Daily Infected':E_Id,'Accumulated Infected':E_Iac,'Deaths':E_D,'UCI/UTI':E_UCI,'VMI':E_VMI}
print(Err)